In [2]:
import cv2
import os
import numpy as np
import re
from google.colab import drive
from scipy.spatial import distance
from scipy.spatial.distance import euclidean, cityblock, mahalanobis
import pandas as pd
from skimage import measure, filters, morphology

drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/mpeg7shapeB'
train_path = '/content/drive/MyDrive/mpeg7shapeB/train'
test_path = '/content/drive/MyDrive/mpeg7shapeB/test'

def preprocess_image(image):
    _, thresholded_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
    return thresholded_image


def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in sorted(os.listdir(folder)):
        try:
            img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                processed_img = preprocess_image(img)
                images.append(processed_img)
                label = filename.split('-')[0]
                labels.append(label)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return images, labels


train_images, train_labels = load_images_from_folder(train_path)
test_images, test_labels = load_images_from_folder(test_path)


def calculate_area(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    return cv2.contourArea(contour)

def calculate_perimeter(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    return cv2.arcLength(contour, True)

def calculate_convexity(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    contour_area = cv2.contourArea(contour)
    return contour_area / hull_area if hull_area != 0 else 0

def calculate_circularity(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    return (4 * np.pi * area) / (perimeter ** 2) if perimeter != 0 else 0

def calculate_rectangularity(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(contour)
    rect_area = w * h
    contour_area = cv2.contourArea(contour)
    return contour_area / rect_area if rect_area != 0 else 0

def calculate_eccentricity(image):
    label_img = measure.label(image > 0)
    regions = measure.regionprops(label_img)
    region = max(regions, key=lambda r: r.area)
    return region.eccentricity


def fourier_descriptor_second(image, num_coefficients=10):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    contour_complex = contour[:, 0, 0] + 1j * contour[:, 0, 1]
    fourier_result = np.fft.fft(contour_complex)
    fourier_magnitude = np.abs(fourier_result)
    return fourier_magnitude[:num_coefficients]

def shape_histogram(image, num_bins=10):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return np.zeros(num_bins)  # Return an empty histogram if no contours are found
    contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(contour)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    distances = [np.linalg.norm([cx - x, cy - y]) for [x, y] in contour.reshape(-1, 2)]
    hist, _ = np.histogram(distances, bins=num_bins, range=(0, max(distances)))
    hist = hist.astype(float)
    hist /= hist.sum()

    return hist

def moment_invariants(image):
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()
    I8 = moments['nu11'] * (moments['nu30'] + moments['nu12'])**2 - (moments['nu03'] + moments['nu21'])**2 - (moments['nu20'] - moments['nu02']) * (moments['nu30'] + moments['nu12']) * (moments['nu03'] + moments['nu21'])
    hu_moments = np.append(hu_moments, I8)

    return hu_moments



def extract_features(image, use_area=True, use_perimeter=True, use_convexity=True, use_circularity=True, use_rectangularity=True, use_eccentricity=True, use_fourier=True, use_histogram=True, use_moments=True, num_fourier_coefficients=200, max_length=50):
    basic_features = []
    fourier_features = []

    # Basic descriptors
    if use_area:
        basic_features.append(calculate_area(image))
    if use_perimeter:
        basic_features.append(calculate_perimeter(image))
    if use_convexity:
        basic_features.append(calculate_convexity(image))
    if use_circularity:
        basic_features.append(calculate_circularity(image))
    if use_rectangularity:
        basic_features.append(calculate_rectangularity(image))
    if use_eccentricity:
        basic_features.append(calculate_eccentricity(image))
    # Histogram features
    if use_histogram:
        basic_features.extend(shape_histogram(image))
    # Moment invariants
    if use_moments:
        basic_features.extend(moment_invariants(image))

    # Fourier descriptors
    if use_fourier:
        fourier_features = fourier_descriptor_second(image, num_coefficients=num_fourier_coefficients)
    combined_features = np.concatenate([basic_features, fourier_features])

    return combined_features


def chi_squared_distance(a, b):
    return np.sum((a - b) ** 2 / (a + b + 1e-10))


def classify(test_feature, train_features, train_labels, dist_func):
    distances = [dist_func(test_feature, train_feature) for train_feature in train_features]
    nearest = np.argmin(distances)
    return train_labels[nearest]


def truncate_features_to_shortest(features):
    min_length = min(len(feature) for feature in features)
    truncated_features = [feature[:min_length] for feature in features]
    return truncated_features


def nearest_neighbor_classification_and_accuracy(dist_func, train_features, test_features, train_labels, VI=None):
    correct_predictions = 0
    for i in range(len(test_features)):
        test_feature = test_features[i]
        true_label = test_labels[i]
        distances = []
        for j in range(len(train_features)):
            train_feature = train_features[j]
            if len(test_feature) > len(train_feature):
                test_feature_truncated = test_feature[:len(train_feature)]
                train_feature_truncated = train_feature
            else:
                test_feature_truncated = test_feature
                train_feature_truncated = train_feature[:len(test_feature)]

            # Calculate the distance
            if dist_func.__name__ == "mahalanobis":
                distance = dist_func(test_feature_truncated, train_feature_truncated, VI)
            else:
                distance = dist_func(test_feature_truncated, train_feature_truncated)

            distances.append(distance)

        nearest_index = np.argmin(distances)
        predicted_label = train_labels[nearest_index]

        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_labels)
    return accuracy


def compute_invertible_covariance_matrix(features, regularization_constant=1e-5):
    cov_matrix = np.cov(features.T)
    regularized_cov_matrix = cov_matrix + np.eye(cov_matrix.shape[0]) * regularization_constant
    return np.linalg.inv(regularized_cov_matrix)

# Define different combinations of feature extraction settings

feature_combinations = [
  {'use_area': True, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': False, 'use_perimeter': True, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': True, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': True, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': True, 'use_eccentricity': False},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': True},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': True, 'use_rectangularity': True, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': True},
  {'use_area': True, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': True, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': True, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': False, 'use_convexity': True, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': True, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': False, 'use_perimeter': True, 'use_convexity': True, 'use_circularity': True, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': True, 'use_convexity': True, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': True, 'use_convexity': False, 'use_circularity': True, 'use_rectangularity': False, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': True, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': True, 'use_eccentricity': False},
  {'use_area': True, 'use_perimeter': True, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': True},
  {'use_area': True, 'use_perimeter': True, 'use_convexity': True, 'use_circularity': True, 'use_rectangularity': True, 'use_eccentricity': True, 'use_fourier': True, 'use_histogram': True, 'use_moments': True},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False, 'use_fourier': True, 'use_histogram': True, 'use_moments': True},
  {'use_area': True, 'use_perimeter': False, 'use_convexity': True, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False, 'use_fourier': True, 'use_histogram': True, 'use_moments': True},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False, 'use_fourier': True, 'use_histogram': False, 'use_moments': False},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False, 'use_fourier': False, 'use_histogram': True, 'use_moments': False},
  {'use_area': False, 'use_perimeter': False, 'use_convexity': False, 'use_circularity': False, 'use_rectangularity': False, 'use_eccentricity': False, 'use_fourier': False, 'use_histogram': False, 'use_moments': True}
]

distance_functions = [
    euclidean,
    cityblock,
    chi_squared_distance,
    mahalanobis
]


def evaluate_combinations(combinations, distance_functions, train_images, train_labels, test_images, test_labels):
    results = []

    for combination in combinations:
        # Extract features for the training and testing datasets
        train_features = [extract_features(img, **combination) for img in train_images]
        test_features = [extract_features(img, **combination) for img in test_images]
        train_features_truncated = truncate_features_to_shortest(train_features)
        VI = None
        if 'mahalanobis' in [func.__name__ for func in distance_functions]:
            train_feature_array = np.array(train_features_truncated)
            VI = compute_invertible_covariance_matrix(train_feature_array)

        # Evaluate each distance function
        for dist_func in distance_functions:
            accuracy = nearest_neighbor_classification_and_accuracy(dist_func, train_features_truncated, test_features, train_labels, VI)
            results.append({
                'Combination': combination,
                'Distance Function': dist_func.__name__,
                'Accuracy': accuracy
            })

    return results


def print_evaluation_results(evaluation_results):
    for result in evaluation_results:
        # Extract and format the combination
        combination = result['Combination']
        enabled_features = [feature for feature, enabled in combination.items() if enabled]
        enabled_features_str = ', '.join(enabled_features)

        # Print the results
        print(f"Combination: {enabled_features_str}")
        print(f"Distance Function: {result['Distance Function']}")
        print(f"Accuracy: {result['Accuracy']:.2f}")
        print("--------------------------------------------------------------------------")


evaluation_results = evaluate_combinations(feature_combinations, distance_functions, train_images, train_labels, test_images, test_labels)

# Call the function to print the results
print_evaluation_results(evaluation_results)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Combination: use_area
Distance Function: euclidean
Accuracy: 0.46
--------------------------------------------------------------------------
Combination: use_area
Distance Function: cityblock
Accuracy: 0.49
--------------------------------------------------------------------------
Combination: use_area
Distance Function: chi_squared_distance
Accuracy: 0.55
--------------------------------------------------------------------------
Combination: use_area
Distance Function: mahalanobis
Accuracy: 0.61
--------------------------------------------------------------------------
Combination: use_perimeter
Distance Function: euclidean
Accuracy: 0.31
--------------------------------------------------------------------------
Combination: use_perimeter
Distance Function: cityblock
Accuracy: 0.31
--------------------------------------------------------------------------
Co